<a href="https://colab.research.google.com/github/soodung2/tensorflow_colab/blob/master/detection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection API Demo

<table align="left"><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Welcome to the [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

> **Important**: This tutorial is to help you through the first step towards using [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) to build models. If you just just need an off the shelf model that does the job, see the [TFHub object detection example](https://colab.sandbox.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb).

# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

### Install

In [ ]:
!pip install -U --pre tensorflow=="2.*"

In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow==1.15


Make sure you have `pycocotools` installed

In [ ]:
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs and install the object_detection package

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd models/research
pip install .

### Imports

In [ ]:
!ls

In [ ]:
cd /content/models/research/object_detection


In [ ]:
!cp -r "/content/gdrive/My Drive/object_detection" "/content/models/research/" 

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

In [ ]:
cd object_detection/

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

Import the object detection module.

In [ ]:
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#전에거에서 추가한거 
from IPython.display import display


In [ ]:
#Env setup

%matplotlib inline
sys.path.append("..")

In [ ]:
#object detection imports
from object_detection.utils import ops as utils_ops
#
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:
cd ..

## Loader

In [ ]:
# What model to download.
MODEL_NAME = 'channel_no5_inference_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('legacy/training/', 'object-detection.pbtxt')

NUM_CLASSES = 1

In [ ]:
#load a (Frozen) tensorflow model into memory


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
#loading label map

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
# Helper code
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Detection

Load an object detection model:

In [ ]:
cd object_detection/

loading labelmap // legacy디렉토리 들어가서 실행해야함


Check the model's input signature, it expects a batch of 3-color images of type uint8: 

*Detection*

In [ ]:
PATH_TO_TEST_IMAGES_DIR='test_images'
TEST_IMAGE_PATH = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image4.jpg')]
#TEST_IMAGE_PATH = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg',format(i)) for i in range(3, 5 ) ]

#TEST_IMAGE_PATH = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, i) for i in os.listdir(PATH_TO_TEST_IMAGES_DIR)]
IMAGE_SIZE = (12, 8)


In [ ]:
with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
      for image_path in TEST_IMAGE_PATH:
       image = Image.open(image_path)

       image_np = load_image_into_numpy_array(image)

       image_np_expanded = np.expand_dims(image_np, axis=0)
       image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

       boxes =  detection_graph.get_tensor_by_name('detection_boxes:0')
       scores = detection_graph.get_tensor_by_name('detection_scores:0')
       classes = detection_graph.get_tensor_by_name('detection_classes:0')
       num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
       # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(image_np)
      plt.show()
      display(Image.fromarray(image_np))
      #cv2.imshow('object detection', cv2.resize(image_np, (800, 600)))

   